# Student's t-test for paired samples in Python

In [1]:
import numpy as np
import timeit
import sys
import os
from subprocess import call, check_output

# Import Python functions from lib
sys.path.insert(0, '../lib_stat')
from lib_stat import avevar, betai

# Compile fortran source code (using a Python script)
os.system('python ../lib_stat/compile_fortran.py');

Data input:

In [2]:
data1 = np.array([70.0, 71.0, 72.0, 73.0, 70.5])
data2 = np.array([70.15, 71.05, 71.95, 72.85, 70.45])

Function definition:

In [3]:
def tptest(data1, data2):
    # Given the arrays "data1" and "data2" of length "n", returns
    # the Student's "t" for paired data and its significance as "prob".

    # Small values of "prob" indicates that the arrays have different
    # means.

    n1 = len(data1)
    n2 = len(data2)
  
    if (n1 != n2):
        print("Data arrays must have the same size")
    else:
        n = n1

    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    cov = 0.0
    for i in range(n):
        cov = cov + (data1[i] - ave1) * (data2[i] - ave2)

    df = n - 1.0
    cov = cov / df

    # Student's t
    t = (ave1 - ave2) / np.sqrt((var1 + var2 - 2.0 * cov) / n)

    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t**2))
    
    return t, prob

#### Calling the Python tptest function:

In [4]:
t, prob = tptest(data1, data2)

print(t, prob)

0.1961161351382784 0.8540797033791303


#### Calling the Fortran tptest function:

In [5]:
fortranOutput = check_output('../lib_stat/fmain_tptest').decode("utf-8").replace('\n','').split(' ')
[float(i) for i in fortranOutput if i]

[0.1961161351382784, 0.8540797033791304]

# Timing against Fortran:

Test function in **Python** and timing:

In [6]:
def speed_test():
    for i in range(1000):
        tptest(data1, data2)
        
%timeit speed_test()

19.9 ms ± 266 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Timing in **Fortran**:

In [7]:
%timeit call(['../lib_stat/fspeed_tptest'])

5.63 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
